In [178]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout
import re
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [179]:
data = pd.read_csv('train.csv')

In [180]:
data

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [181]:
data.shape

(20800, 5)

In [182]:
data = data.dropna()

In [183]:
X = data.drop('label', axis = 1)

In [184]:
X

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [185]:
y = data['label']

In [186]:
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64

In [187]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [188]:
X.shape

(18285, 4)

In [189]:
y.shape

(18285,)

In [190]:
message = X.copy()

In [191]:
message

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [192]:
message.reset_index(inplace= True)

In [193]:
ps = PorterStemmer()
corpus = []
for i in range(0, len(message)):
    review = re.sub('[^a-zA-Z]', ' ', message['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for  word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [194]:
voc_size = 10000

In [195]:
one_hot_rep = [one_hot(word, voc_size) for word in corpus]

In [196]:
one_hot_rep

[[1043, 9931, 5442, 5270, 257, 5800, 4367, 32, 8620, 8704],
 [1928, 7196, 1518, 3223, 3781, 9805, 2385],
 [4896, 7872, 94, 1242],
 [209, 145, 7633, 6717, 2347, 1841],
 [4762, 3781, 6092, 8490, 1476, 7066, 3781, 208, 9915, 1777],
 [2019,
  2964,
  6834,
  1824,
  2310,
  1307,
  6146,
  2932,
  1838,
  870,
  4911,
  9453,
  3923,
  7840,
  2385],
 [4139, 4612, 5387, 4721, 7116, 3826, 5280, 1912, 3841, 9117, 8985],
 [3650, 6975, 4890, 6111, 338, 6693, 1307, 7982, 3841, 9117, 8985],
 [6923, 2578, 68, 5320, 4468, 1965, 960, 2410, 1307, 1930],
 [8652, 3372, 6016, 6610, 8530, 3061, 2040, 9641],
 [9861, 8738, 9574, 5016, 6773, 3987, 9460, 3590, 7649, 3678, 4861],
 [6717, 6704, 257, 1965, 1307, 338],
 [2474, 3674, 3627, 2009, 4384, 6333, 7367, 8024, 2228],
 [734, 3117, 1255, 5683, 719, 7172, 462, 3841, 9117, 8985],
 [9564, 9168, 8537, 277, 5628, 3841, 9117, 8985],
 [4669, 4262, 9891, 3529, 1616, 5877, 8124, 560, 593, 6102],
 [2596, 9038, 7196],
 [8435, 8722, 7548, 5574, 1307, 2367, 778, 2385]

In [197]:
max_sent = 20

In [198]:
padding = pad_sequences(one_hot_rep,  padding = 'pre',  maxlen = max_sent)
padding

array([[   0,    0,    0, ...,   32, 8620, 8704],
       [   0,    0,    0, ..., 3781, 9805, 2385],
       [   0,    0,    0, ..., 7872,   94, 1242],
       ...,
       [   0,    0,    0, ..., 3841, 9117, 8985],
       [   0,    0,    0, ..., 9411, 6837, 4730],
       [   0,    0,    0, ..., 2253, 9740, 6626]])

In [199]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=max_sent))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 20, 40)            400000    
_________________________________________________________________
bidirectional_5 (Bidirection (None, 200)               112800    
_________________________________________________________________
dropout_4 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 201       
Total params: 513,001
Trainable params: 513,001
Non-trainable params: 0
_________________________________________________________________
None


In [200]:
X_final = np.array(padding)

In [201]:
y_final = np.array(y)

In [202]:
X_train,X_test,y_train,y_test = train_test_split(X_final,y_final,test_size = 0.33, random_state = 10)

In [203]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=40)

Epoch 1/10
307/307 [==============================] - 9s 28ms/step - loss: 0.0076 - accuracy: 0.9976 - val_loss: 0.1053 - val_accuracy: 0.9793
Epoch 2/10
307/307 [==============================] - 8s 25ms/step - loss: 0.0076 - accuracy: 0.9969 - val_loss: 0.1064 - val_accuracy: 0.9788
Epoch 3/10
307/307 [==============================] - 8s 25ms/step - loss: 0.0055 - accuracy: 0.9982 - val_loss: 0.1113 - val_accuracy: 0.9758
Epoch 4/10
307/307 [==============================] - 8s 26ms/step - loss: 0.0058 - accuracy: 0.9978 - val_loss: 0.1217 - val_accuracy: 0.9723
Epoch 5/10
307/307 [==============================] - 7s 24ms/step - loss: 0.0018 - accuracy: 0.9995 - val_loss: 0.1393 - val_accuracy: 0.9733
Epoch 6/10
307/307 [==============================] - 8s 25ms/step - loss: 2.6585e-04 - accuracy: 1.0000 - val_loss: 0.1500 - val_accuracy: 0.9737
Epoch 7/10
307/307 [==============================] - 7s 24ms/step - loss: 5.1080e-05 - accuracy: 1.0000 - val_loss: 0.1555 - val_accuracy

In [204]:
y_pred = model.predict_classes(X_test)

C:\Users\USER\.conda\envs\carprediction\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [205]:
y_pred

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [206]:
confusion_matrix(y_test,y_pred)

array([[3437,    0],
       [2589,    9]], dtype=int64)

In [207]:
accuracy_score(y_test,y_pred)

0.5710024855012428

In [208]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.57      1.00      0.73      3437
           1       1.00      0.00      0.01      2598

    accuracy                           0.57      6035
   macro avg       0.79      0.50      0.37      6035
weighted avg       0.76      0.57      0.42      6035

